## Project 2 - Casimir Funk



In [1]:
!pip install -r requirements.txt
import pandas as pd

### 1.1 #A Description of population of interest.

Population of San Francisco and comparing the difference in minimum cost diets of people across different diet groups, such as vegetarian, pescatarian, vegan, meat eater, etc.

### 1.2 #A Dietary Reference Intakes

Write a function that takes as arguments the characterists of a person (e.g. age, sex) and returns a pandas.Series of Dietary Reference Intakes (DRI's) or "Recommended Daily Allowances" (RDA) of a variety of nutrients appropriate for your population of interest.

In [2]:
def DRI(age, sex):
    bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:,2:]
    bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition').iloc[:,2:]
    combined_df = pd.concat([bmin,-bmax])
    if sex == "Male":
        if age <=3:
            return combined_df.loc[:, 'C 1-3']
        if age>=4 and age<=8:
            return combined_df.loc[:, 'M 4-8']
        if age>=9 and age<=13:
            return combined_df.loc[:, 'M 9-13']
        if age>= 14 and age<=18:
            return combined_df.loc[:,'M 14-18']
        if age>=19 and age <= 30: 
            return combined_df.loc[:, 'M 19-30']
        if age>=31 and age<=50:
            return combined_df.loc[:, 'M 31-50']
        if age>=51:
            return combined_df.loc[:, 'M 51+']
    if sex == "Female":
        if age <=3:
            return combined_df.loc[:, 'C 1-3']
        if age>=4 and age<=8:
            return combined_df.loc[:, 'F 4-8']
        if age>=9 and age<=13:
            return combined_df.loc[:, 'F 9-13']
        if age>= 14 and age<=18:
            return combined_df.loc[:,'F 14-18']
        if age>=19 and age <= 30: 
            return combined_df.loc[:, 'F 19-30']
        if age>=31 and age<=50:
            return combined_df.loc[:, 'F 31-50']
        if age>=51:
            return combined_df.loc[:, 'F 51+']
DRI(25, 'Male')

Nutrition
Energy                            2400.0
Protein                             56.0
Fiber, total dietary                33.6
Folate, DFE                        400.0
Calcium, Ca                       1000.0
Carbohydrate, by difference        130.0
Iron, Fe                             8.0
Magnesium, Mg                      400.0
Niacin                              16.0
Phosphorus, P                      700.0
Potassium, K                      4700.0
Riboflavin                           1.3
Thiamin                              1.2
Vitamin A, RAE                     900.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      90.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)          120.0
Zinc, Zn                            11.0
Sodium, Na                       -2300.0
Name: M 19-30, dtype: float64

### 1.3 #A Data on prices for different foods (Google Sheets)

API user and keys:

In [3]:
user = "project2-casimir-funk"

# API key for Gov
apikey = "V0mOAdVrSineT4d2VTMTJUNPAsBTnjzUAU6e6H6V" # inIyO1begWSRqsYtxS7m6p09PSyq7Qiw7fxzV2qN"

# File with private keys for relevant service account to authenticate
# and access google spreadsheets
!gpg --passphrase "casimir" -d project2-casimir-funk-306520-ae3703c35882.json.gpg > project2-casimir-funk-306520-ae3703c35882.json

serviceacct = {'project2-casimir-funk':'project2-casimir-funk-306520-ae3703c35882.json'}
sheet_url = "https://docs.google.com/spreadsheets/d/1XqnsamAhXq2xnktB_rJ8h8ovvpFQdWWcDkx_1Yrfqdg/edit#gid=0"


gpg: AES256 encrypted data
gpg: encrypted with 1 passphrase


In [4]:
from eep153_tools import read_sheets

# Accessing google sheets and convert to pandas.dataframe
data = read_sheets(key=sheet_url, json_creds=serviceacct[user], sheet="master")
headers = data.iloc[0]
df  = pd.DataFrame(data.values[1:], columns=headers)
df

,Category,Food,Quantity,Units,Price,FDC,link to price
0,Veggies,Broccoli,1.00,pounds,2.29,1439817,https://www.safeway.com/shop/produ-details.184...
1,,Onions (Yellow),7.00,ounces,0.75,1103364,https://www.target.com/p/yellow-onion-each/-/A...
2,,Spinach,9.00,ounces,2.29,1377964,https://www.target.com/p/steam-in-bag-spinach-...
3,,Russet Potatoes,9.00,ounces,0.95,1357648,https://www.target.com/p/russet-potatoes-each/...
4,,Sweet Potatoes,3.00,pounds,2.99,1387790,https://www.target.com/p/sweet-potatoes-3lb-ba...
5,,Canned Tomatoes,28.00,ounces,1.79,170051,https://www.target.com/p/hunt-39-s-100-natural...
6,,Carrots,1.00,pounds,1.29,1321717,https://www.target.com/p/baby-cut-carrots-1lb-...
7,,Green Cabbage,3.40,pounds,1.79,1447540,https://www.target.com/p/green-cabbage-each/-/...
8,,Butternut Squash,12.00,ounces,2.49,1352434,https://www.target.com/p/frozen-butternut-squa...
9,Grains/Legumes,Brown Rice,1.00,pounds,0.95,449460,https://www.target.com/p/long-grain-brown-rice...


In [5]:
import fooddatacentral as fdc

df['Quantity'] = pd.to_numeric(df['Quantity'], downcast="float") 
df['Price'] = pd.to_numeric(df['Price'], downcast="float") 

df['FDC Quantity'] = df[['Quantity', 'Units']].T.apply(lambda x: fdc.units(x['Quantity'], x['Units']))

df['FDC Price'] = df['Price']/df['FDC Quantity']
df.dropna()
prices = df.groupby('Food')['FDC Price'].min()
prices

Food
10 pc McNuggets                     3.6100627491309205 / hectogram
Anchovies                           2.3970260063559543 / hectogram
Apples                             0.22046226218487755 / hectogram
Avocado                             0.2020904033526449 / hectogram
Bananas                            0.20458897342058885 / hectogram
Beef                                 1.541472096828818 / hectogram
Bell Pepper                         0.9898755067502929 / hectogram
Big Mac                              2.882254050016064 / hectogram
Black Beans                        0.11151123672060076 / hectogram
Broccoli                            0.5048585719934017 / hectogram
Brown Rice                          0.2094391464475187 / hectogram
Butternut Squash                    0.7319347132571161 / hectogram
Canned Beans                       0.11151123672060076 / hectogram
Canned Salmon                       1.5449995737594677 / hectogram
Canned Tomatoes                    0.22550139480055026 / 

### 1.4 #A Nutritional content of different foods

For all the foods you're considering you'll need to be able to describe their nutritional content, in terms that allow you to compare with recommended daily allowances. We'll provide points to US government recommendations, but many other countries have their own recommendations, which may be better suited to their populations.

In [ ]:
import fooddatacentral as fdc

food_to_nutrients = {}
for food in df['Food'].tolist():
    try:
        fdc_id = int((df[df['Food'] == food].FDC).iloc[0])
        food_to_nutrients[food] = fdc.nutrients(apikey, fdc_id).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))
        
pd.set_option('display.max_rows', 50)
nutritional_df = pd.DataFrame(food_to_nutrients, dtype=float)
nutritional_df

In [ ]:
prices = prices.dropna()
selection = list(set(prices.index.tolist()).intersection(nutritional_df.columns.tolist()))
prices = prices[selection]
prices

In [ ]:
df = pd.DataFrame(DRI(25, 'Male'))
df

### 1.5 #A Solution

What is the minimum cost diet for the population you're concerned with? How much does it cost, and of what does it consist? How does it vary with age, sex, and level of activity?

### 1.6 #B Is your solution edible?

Turn your minimum cost diet into a meal. Share the recipe on piazza. If making your diet palatable requires adding additional ingredients (e.g. salt or other spices) besure to document this and account for it in the overall cost. (You may want to take pictures or video of preparation and eating). Whose meal is least costly? Whose is most tasty?

### 1.7 #B Meal Reviews

Get the recipe for the meal from some other group(s), and prepare the meal according to their recipe. Provide a review (follow-up to the piazza post with the recipe). How much did the ingredients cost? Would you recommend it to others? Eat it again? Eat it exclusively?

### 1.8 #C Sensitivity of Solution

In general, your solution will depend on relative prices of different kinds of foods. And these prices can be quite volatile; for example, the prices of fresh fruits and vegetables usually exhibit a great deal of seasonal variation. If the price of a particula rkind of food increases by x%, how much does price of diet change, and how does the composition of the diet change? Try and find an nice way to visualize these changes.

### 1.9 #B What is the total cost for population of interest?

You've been interested in a particular population. It is likely to consist of people of different sexes, ages, and levels of activity. How much food would be required to feed the entire population of interest? At what cost? (You may want to go back to the Population Project and look at counts of people by age and sex for different countries).

### 1.10 #B Review of Your Code

### 1.11 #B Review of Another Group's Code

### 1.12 #A Presentation